# Random Forest Model

In [1]:
import pre_ml

In [2]:
df = pre_ml.data()
df.head(10)

Requesting data to datos abiertos Mexico
Getting zip raw data into directory, will delete soon
Raw data deleted. If you specified filename, clean data will be saved in data directory


,entidad_res,municipio_res,CVE_MUN,embarazo,edad,diabetes,epoc,asma,inmusupr,hipertension,cardiovascular,obesidad,tabaquismo,hospitalizado,muertos,pobreza,Densidad_pob,medicos,camas_hosp,enfermeras
0,25,6,25006,0.0,55,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1,1,31.2,152.715706,2.637944,10.489463,30.502528
1,27,8,27008,0.0,103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,66.2,53.974231,3.289146,2.292435,5.282568
2,26,55,26055,0.0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,36.9,23.316425,2.657236,3.913384,13.431120
3,15,104,15104,0.0,70,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,1,28.7,9800.971628,0.700561,8.129146,18.849045
4,9,7,9007,0.0,36,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,36.4,16028.524764,2.170140,7.325600,17.785234
5,15,33,15033,0.0,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,37.6,10709.607425,1.147706,4.807484,14.258494
6,5,14,5014,0.0,92,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,1,68.7,4.995184,2.389511,10.888391,27.416795
7,9,7,9007,0.0,35,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,36.4,16028.524764,2.170140,7.325600,17.785234
8,30,193,30193,0.0,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,31.3,2537.821317,4.131322,16.110560,42.158624
9,9,14,9014,0.0,62,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0,3.2,16243.745318,6.156216,28.636779,83.143497
